# UWB 자율주행 물류 로봇 통합 관리 시스템

In [ ]:
robot_list = [[1, 'locus_bot.jpg', 'Locus_bot', '현재 로봇 6대 운행중', '없음'],
                 [2, 'chuck.jpg', 'Chuck', '현재 로봇 6대 운행중', '없음'],
                 [3, 'go_cart_250.jpg', 'gocart_2500', '현재 로봇 6대 운행중', '없음'],
                 [4, 'M3_1000.jpg', 'M3_1000', '현재 로봇 6대 운행중', '없음'],
                 [5, 'hanhasf_logistics.jpg', 'hanhasf_logistics', '현재 로봇 6대 운행중', '없음'],
                 [6, 'TES.jpg', 'TES', '현재 로봇 6대 운행중', '없음']
             ]
smartfarm_list = [[i, f'smartfarm0{i}.png', f'스마트명{i}', '현재 로봇 6대 운행중', '없음'] for i in range(1, 6)]
sf_info = [[i, f'smartfarm0{i}.png', f'스마트명{i}','현재 로봇 10대 운행중', '없음', 
          {'Locus_bot':[i*6-5, '작업중'],
           'Chuck':[i*6-2, '작업중'],
           'gocart_2500' :[i*6-3, '대기중'],
           'M3_1000' : [i*6, '작업중'],
           'hanhasf_logistics' :[i*6-1, '대기중'],
           'TES':[i*6-4, '점검중']
          }] for i in range(1, 6)]
r_info = {'Locus_bot': [[i, '작업중', f'sf_{i*3-2}'] for i in range(1, 7)],
           'Chuck':[[i, '대기중', f'sf_{i*3-2}'] for i in range(7, 13)],
           'gocart_2500' :[[i, '작업중', f'sf_{i*3-2}'] for i in range(13,19)],
           'M3_1000' : [[i, '점검중', f'sf_{i*3-2}'] for i in range(19, 25)],
           'hanhasf_logistics' :[[i, '작업중', f'sf_{i*3-2}'] for i in range(25, 31)],
           'TES':[[31, '작업중', 1,(305, 400)], [32, '대기중', 3,(320, 240)], [33, '점검중', 2,(25, 38)], [34, '작업중', 1,(325, 120)], [35, '작업중', 1,(5, 400)], [36, '대기중', 2,(10, 500)]]
          }

member_list = (('dct', '1234', 'm'), ('richfarmer','7777', 'u'))

In [ ]:
[m for m in member_list if m[:2] == ('dct', '1234')][0]

### 로그인

In [ ]:
from DB import DAO

# 로그인
def login_ck(mlogin,mid,mpw='') :  # login_ck(로그인 경로, 아이디, 비밀번호)
    conn, cursor = conn_db()  # DB 접속
    
    # Diamond 홈페이지 로그인 시,
    if mlogin == 'D' :
        query = """SELECT mb_id, mb_signup_type, mb_type FROM tbl_member 
                        where mb_signup_type='D' and mb_id= :1 and mb_pw= :2 """
        cursor.execute(query, [mid, mpw])  # 쿼리문 실행 
#         cursor.execute(f"""SELECT mb_id, mb_signup_type, mb_type FROM tbl_member 
#                         where mb_signup_type='D' and mb_id='{mid}'and mb_pw='{mpw}'""")    
    
    # 간편 로그인 시,
    else :
        query = "SELECT mb_id, mb_signup_type, mb_type FROM tbl_member where mb_signup_type= :1 and mb_id= :2"
        cursor.execute(query, [mlogin, mid])
#         cursor.execute(f"SELECT mb_id, mb_signup_type, mb_type FROM tbl_member where mb_signup_type='{mlogin}' and mb_id='{mid}'")
    data = cursor.fetchone()  # 조회 결과(한 튜플) 가져오기
    close_db(conn, cursor)  # DB 접속 해제
    
    # 유효한 ID, PW 입력시, session 갱신
    if data != None :
        session['user_id'] =  data[0]
        session['login_by'] = data[1]
        session['manager'] = data[2]
        return redirect(url_for('index_info', info='001777'))  # 로그인 성공 페이지로 이동
    
    # 유효하지 않은 ID, PW 입력시,
    else :
        return redirect(url_for('login_info', info='001666'))  # 로그인 실패 페이지로 이동

    
# 회원정보 가져오기
def select_all(mlogin,mid) :  # select_all(로그인 경로, 아이디)
    conn, cursor = conn_db()  # DB 접속
    
    query = """SELECT mb_name, mb_id, mb_phone, mb_email, mb_signup_type, mb_type FROM tbl_member 
                    where mb_signup_type= :1 and mb_id= :2"""
    cursor.execute(query, [mlogin, mid])  # 쿼리문 실행 
    data = cursor.fetchone()  # 조회 결과(한 튜플) 가져오기
    close_db(conn, cursor)  # DB 접속 해제
    
    if data:
        return data
    else :
        return render_template('/404')


In [ ]:
def temp_login (mid, mpw) :
    for m in member_list :
        if m[:2] == (mid, mpw) :
            session['user_id'] = m[0]
            session['login_by'] = m[1]
            session['manager'] = m[2]
        return redirect(url_for('index_info', info='001777'))  # 로그인 성공 페이지로 이동
    
    # 유효하지 않은 ID, PW 입력시,
    else :
        return redirect(url_for('login_info', info='001666'))  # 로그인 실패 페이지로 이동

## 회원가입

### DB 접속, 해제

In [1]:
import cx_Oracle
import os

# DB 접속
def conn_db() :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기    
    conn = cx_Oracle.connect('dct','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    return conn, cursor

# DB 접속 해제
def close_db(conn, cursor) :
    cursor.close()
    conn.close()

## 회원가입

In [2]:
# 아이디 중복 체크
def check_id(M_ID) :
    conn, cursor = conn_db()  # DB 접속
    
    query = f"""SELECT COUNT(*) FROM MEMBERS WHERE M_ID = :1"""
    cursor.execute(query, [M_ID])  # 쿼리문 실행 
    data = cursor.fetchone()  # 조회 결과(한 튜플) 가져오기
    close_db(conn, cursor)  # DB 접속 해제
    
    if data[0] == 0:
        return jsonify({'message':'success', 'result': True})
    else :
        return jsonify({'message':'success', 'result': False})
    
# 회원가입
def signup_member(M_NAME, M_ID, M_PW, M_TEL) :
    conn, cursor = conn_db()  # DB 접속
    
    query = f"""INSERT INTO MEMBERS (M_NAME, M_ID, M_PW, M_TEL) VALUES (':1', ':2', ':3', ':4')"""
    cursor.execute(query, [M_NAME, M_ID, M_PW, M_TEL])  # 쿼리문 실행 
    
    query = f"""SELECT COUNT(*) FROM MEMBERS WHERE M_ID = ':1'"""
    cursor.execute(query, [M_ID])  # 쿼리문 실행 
    
    data = cursor.fetchone()  # 조회 결과(한 튜플) 가져오기
    print(data[0])
    close_db(conn, cursor)  # DB 접속 해제
    
    if data[0] == 1:
        return jsonify({'message':'success', 'result': True})
    else :
        return jsonify({'message':'fail', 'result': False})

## 로그인

In [3]:
# 로그인
def login_member(M_ID, M_PW) :
    conn, cursor = conn_db()  # DB 접속
    
    query = f"""SELECT M_NO, M_ID FROM MEMBERS WHERE M_ID = :1 AND M_PW = :2"""
    cursor.execute(query, [M_ID, M_PW])  # 쿼리문 실행 
    data = cursor.fetchone()  # 조회 결과(한 튜플) 가져오기
    print(data)
    close_db(conn, cursor)  # DB 접속 해제
    
    if data != None:
        session['user_no'] = data[0]
        session['user_id'] = data[1]
        return jsonify({'message':'success', 'result': True})
    else :
        return jsonify({'message':'success', 'result': False})

In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for, jsonify
from markupsafe import escape
from datetime import timedelta, datetime
import random

app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=1) # 로그인 지속 시간

token, info, cage_list, cage_lifo = '', '', '', ''


############# 회원 가입 #############
@app.route('/signup')  # 회원 가입
def signup() :
#     if len(session) :
#         return redirect(url_for('index_info', info='705001'))
#     else :
    return render_template('signup.html', title="회원 가입")


@app.route('/signup/idck' , methods=["POST"])  # 아이디 중복 체크
def ck_id() :
    M_ID = request.get_json()['M_ID']
    
    return check_id(M_ID)


@app.route('/signup/result', methods=["POST"])  # 회원 가입 결과
def signup_result() :
    if request.method == 'POST' :
        M_NAME = request.form['M_NAME']
        M_ID = request.form['M_ID']
        M_PW = request.form['M_PW']
        M_TEL = request.form['M_TEL']
        return signup_member(M_NAME, M_ID, M_PW, M_TEL)
    return render_template('404.html', title="잘못된 접근")

######### 로그인 #########
@app.route('/') # 로그인 
def login() :
    if len(session) :
        return redirect(url_for('index_info', info='705001'))
    return render_template('login.html', title="로그인")

@app.route('/<info>')
def login_info(info) :  #로그인 + 이전 페이지 결과
    return render_template('login.html', info=info, title="로그인")


@app.route('/login/result', methods=['POST']) # 로그인 + 입력값 체크
def login_result() :
    if request.method == 'POST' :
        M_ID = request.form['M_ID']
        M_PW = request.form['M_PW']
        return login_member(M_ID, M_PW)
    return render_template('404.html', title="잘못된 접근")
    

    
#!!!!!!!!!!!! session 접근 권한 설정 해야 함!!!!!!!!!!!!!!!!
######### 홈 #########
@app.route('/home')  # 홈
def main() :
    return render_template('home.html', title="홈")



######### 관리 및 설정 #########
@app.route('/set')  # 관리 및 설정
def setting() :
    return render_template('setting.html', title="관리 및 설정")


# ######### 농장 #########
# @app.route('/smartfarm')
# def smartfarm() :
    
#     smartfarm_robot = {'what' : '스마트팜',
#                       'list' : smartfarm_list,
#                       'image' : 'smartfarm.png'}
#     return render_template('smartfarm_robot.html', session=session, smartfarm_robot=smartfarm_robot)

# # 농장 정보 확인
# @app.route('/smartfarm/sf_<sf_no>')
# def smartfarm_show(sf_no) :
#     for sf in sf_info :
#         if sf[0] == int(sf_no) :
#             return render_template('smartfarm_show.html', session=session, sf_state=sf)

# ######### 로봇 #########
# @app.route('/robot')
# def robot() :
#     smartfarm_robot = {'what' : '로봇',
#                       'list' : robot_list,
#                       'image' : 'robot.png'}
#     return render_template('smartfarm_robot.html', session=session, smartfarm_robot=smartfarm_robot)

# # 위치 확인- 첫화면
# @app.route('/robot/<r_model>')
# def robot_loc0(r_model) :
#     return render_template('robot_loc.html', session=session, r_name=r_model, r_list=r_info[r_model], r_state=None)


# # 위치 확인 - 모델 번호로 검색
# @app.route('/robot/<r_model>&<r_no>')
# def robot_loc(r_model,r_no) :
#     for r in r_info[r_model] :
#         if r[0] == int(r_no) :
#             return render_template('robot_loc.html', session=session, r_name=robot_list[int(r_no)][2], r_list=r_info[r_model], r_state=r)
# #     return redirect(url_for())

# ######### 직원 #########
# @app.route('/employee')
# def employee() :
#     return render_template('employee.html', session=session)

if __name__ == '__main__' :
    app.run(host="192.168.35.204", port=2048)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.35.204:2048/ (Press CTRL+C to quit)
192.168.35.204 - - [02/Dec/2022 19:40:26] "GET / HTTP/1.1" 200 -
192.168.35.204 - - [02/Dec/2022 19:40:26] "GET /favicon.ico HTTP/1.1" 200 -
192.168.35.204 - - [02/Dec/2022 19:40:27] "GET /signup HTTP/1.1" 200 -
192.168.35.204 - - [02/Dec/2022 19:40:27] "GET /favicon.ico HTTP/1.1" 200 -
192.168.35.204 - - [02/Dec/2022 19:40:33] "POST /signup/idck HTTP/1.1" 200 -
192.168.35.204 - - [02/Dec/2022 19:40:43] "GET /static/bootstrap.bundle.min.js.map HTTP/1.1" 200 -
[2022-12-02 19:40:47,581] ERROR in app: Exception on /signup/result [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_